# Project Description

**Purpose** <br>
This project consists of analyzing several datasets to help in the research efforts of a new Chicago ride share company. The analysis will provide insights on trends in taxi service including popular service providers and neighborhoods. 

**Datasets** <br>
The datasets used are from data extracted from a website with weather condition information as well as SQL database queriers. 

The first dataset includes taxi companies names and the number of rides they completed on November 15-16, 2017. The secodn dataset used contains drop-off locations in Chicago neighborhoods where rides ended and the average number of rides that ended in each neighborhood in November 2017.

**Analysis** <br>
The analysis involves creating simple visualizations to identify the most frequently used Taxi services in Chicago and the top 10 neighborhoods in terms of drop-offs. These visualizations will compare different taxi service providers using bar charts. 

The final part of this project includes hypothesis testing using the data from /datasets/project_sql_result_07.csv.
This dataset includes information about rides from the Loop to O'Hare International Airport, including pickup date and time, weather conditions at the time of pickup, and ride duration in seconds. 

The hypothesis to be tested is: "The average duration of rides from the Loop to O'Hare International Airport changes on rainy Saturdays."

Conducting this test will determine whether there is a significant difference between average ride times to the Airport on Saturdays depending on the weather condition. This may be helpful for planning the amount of rides or drivers set out on Saturdays by the taxi service in order to provide the best service and expectation

# Step 4: Exploratory Data Analysis

## Import Data 

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
from scipy import stats
import os

# Set the working directory (Was having issues with this since I had a different notebook I was working on at the start)
os.chdir('/home/jovyan/work/sprint_6_project')
print(os.getcwd())

: 

## Read Data

In [ ]:
# Read the CSV files 
company_trips = pd.read_csv('moved_project_sql_result_01.csv')
average_trip_location = pd.read_csv('moved_project_sql_result_04.csv')


: 

In [ ]:
#Viewing 10 representative rows of data 
company_trips.sample(10)


: 

Data types appear correct but some entries seem to have numeric values at the beginning, such as "1085 - 72312 N and W Cab Co". This could be an id from a different table that was mergered incorrectly but it is useless as is, so we will just delete these values from all company_name values. 

In [ ]:
# Using regular expression to remove numeric values from company names for clarity
company_trips['company_name'] = company_trips['company_name'].str.replace(r'^\d+\s*-\s*|\d+\s*', '')
company_trips['company_name'].sample(10)

: 

In [ ]:
#Viewing 10 representative rows of data 
average_trip_location.sample(10)


: 

## Ensure Correct Data Types 

In [ ]:
# Viewing basic data information 
company_trips.info()


: 

Data types appear correct but further investigation is necessary to ensure there are no abnmormalities.

In [ ]:
# Viewing basic data information 
average_trip_location.info()

: 

Data types appear correct but further investigation is necessary to ensure there are no abnmormalities.

In [ ]:
company_trips.describe()


: 

All values seem normal, showing the taxi companies with the least amount of trips completed at the min() value of 2 trips and the company with the most at the max() value of 19,558 trips completed. 

In [ ]:
average_trip_location.describe()


: 

All values seem normal, showing the least and most popular drop off locations at the min() value of 1.8 trips on average and max() value of 10,727 trips on average at the most popular destination.  

The only issue is that the float is not the easiest to read as is. I debated removing changing it to integers but will round the decimals up instead. 

In [ ]:
# Rounding the decimal points for readability 
average_trip_location['average_trips'] = average_trip_location['average_trips'].round(2)
average_trip_location


: 

## Data Visualizations 


**Total Trips Completed by Each Taxi Company**


In [ ]:
# Creates bar graph for total trips by company 
fig = px.bar(company_trips, x='trips_amount', y='company_name', 
             title='Total Taxi Trips by Company',
             category_orders={"company_name": company_trips['company_name'][::-1]})
fig.update_layout(width=800, height=600, xaxis_title='Total Trips Completed', yaxis_title='Company Name', xaxis_tickangle=0)
fig.show()


: 

The top taxi service, Flash Cab has a great lead on total trips compeleted, showing that it is clearly very seasoned in the field and has a large fleet and established presence compared to all other services in the area. 

There are only a very few dominant players in the Chicago taxi business with only the top 12 companies appear to have completed a total of at least 100 trips, which makes it clear that there are only a handful of extremely popular services which most people use in the Chicago area. 



### Top Taxi Services 

In [ ]:
# Top 10 taxi services by total trips completed 
top_10_company = company_trips.sort_values(by='trips_amount', ascending=False)
top_10_company = top_10_company .head(10)
top_10_company 

: 

<br> 

**Top 10 Taxi Companies by Total of Trips Completed** 

In [ ]:
# Creates bar graph for top 10 companies
fig = px.bar(top_10_company , x='trips_amount', y='company_name', 
             title='Top 10 Taxi Companies',
             category_orders={"company_name": company_trips['company_name'][::-1]})
fig.update_layout(xaxis_title='Total Trips Completed', yaxis_title='Company Name', xaxis_tickangle=0)
fig.show()

: 

Again, it appears clear that 'Flash Cab' is in it's own league, while the rest of the taxi services are close competition to eachother. 

### Top 10 Neighborhood Destinations



**Top 10 Neighborhoods by Number of Dropoffs**
<br> 


In [ ]:
# Top 10 drop-off locations by neighborhood 
top_10_dropoff = average_trip_location.sort_values(by='average_trips', ascending=False)
top_10_dropoff= top_10_dropoff.head(10)
top_10_dropoff

: 

In [ ]:
# Creates bar graph for top 10 drop offs
fig = px.bar(top_10_dropoff, x='average_trips', y='dropoff_location_name', 
             title='Top 10 Neighborhood Destinations',
             category_orders={"dropoff_location_name": top_10_dropoff['dropoff_location_name'][::-1]})
fig.update_layout(width=800, height=600, xaxis_title='Average Trips Completed', yaxis_title='Drop Off Locations', xaxis_tickangle=0)
fig.show()


: 

**Top Neighborhood Destinations** 

#1  Loop

#2 River North

#3 Streeterville

#4 West Loop

<br>

After the top 4 destinations listed above, there is a significant drop in trip totals for all following neighborhoods. This means that the top 4 are especially popular. It was suprising to see that O'Hare- the airport destination is not higher on the list, but this might be because certain commuter locations may be more popular for everyday taxi rides. 

# Step 5: Testing hypotheses (Python)

The last dataset used will be the result of the final SQL query and contains data on rides from the Loop to O'Hare International Airport. 

<br>

The table's field values are:

start_ts- This is the pickup date and time

weather_conditions- The weather conditions at the moment the ride started

duration_seconds- The ride duration in seconds

## Importing and Data Cleaning  

In [ ]:
# Imports data 
loop_to_airport = pd.read_csv('/datasets/project_sql_result_07.csv')
loop_to_airport.sample(10)

: 

From this sample, it appears that the column 'start_ts' only shows the hour of pickup, although the current format suggests that it has the information up to seconds. The column may not be set to date time. The name of the column could also be changed for clarity. 

The 'weather_conditions' column looks great so far. 

The 'duration_seconds'column could be simplified to just integers instead of float.

In [ ]:
# Check dataframe information 
loop_to_airport.info()

: 

In [ ]:
# Making changes to dataframe columns
loop_to_airport['duration_seconds'] = loop_to_airport['duration_seconds'].astype(int)

# Renaming and changing data type to show clear pickup time
loop_to_airport = loop_to_airport.rename(columns={'start_ts': 'pickup_date_time'})
loop_to_airport['pickup_date_time'] = pd.to_datetime(loop_to_airport['pickup_date_time']) 

: 

In [ ]:
# Confirming changes 
loop_to_airport.info()

: 

### Filtering Data by Day of Week = Saturday 

In this section we will only work with data from the Loop to Airport Saturdays in November 2017. 
<br>
Those Saturdays are Novemeber: the 4th, 11, 18 and 25.

In [ ]:
# Filters for Saturdays in November 2017
saturday_rides = loop_to_airport[loop_to_airport['pickup_date_time'].dt.dayofweek == 5]
saturday_rides

: 

In [ ]:
# Checking that the Saturday dataframe is clear 
saturday_rides.sample(10)

: 

### Preparing Data for Tests 

In [ ]:
# Splitting data up into bad and good weather by  weather_condition and duration_seconds 
no_rain_saturdays = saturday_rides[saturday_rides['weather_conditions'] == 'Good']['duration_seconds']

rain_saturdays = saturday_rides[saturday_rides['weather_conditions'] == 'Bad']['duration_seconds']

: 

Identifying the variance first to determine the alpha level for testing:

In [ ]:
# Checking Indivudal variances
no_rain_var = no_rain_saturdays.var()
rain_var = rain_saturdays.var()

print(f'The variance of Saturdays with no rain is {no_rain_var}')
print(f'The variance of rainy Saturdays is: {rain_var}')
# Levene's test for variance comparison
levene_test = stats.levene(rain_saturdays, no_rain_saturdays)
levene_test

: 

Although the variance is higher for Saturdays with no rain, the difference between the two variances doesn't seem significant. 

The p-value of the Levene test is larger than the typical significance level of 0.05. Therefore, we can assume equal variances for now during testing.

In [ ]:
# Double checking 
# Finding the mean and standard deviation of the dataset to compare with the variance 
no_rain_saturdays.describe()

: 

In [ ]:
# Finding the mean and standard deviation of the dataset to compare with the variance 
rain_saturdays.describe()

: 

#### Comparing mean, standard deviation (std), and variance 
Comparing the mean, standard deviation (std), and variance for rides from the Loop to O'Hare International Airport is an extra step we can take to determine significance level for testing. 

**Saturdays without rain:**<br>
Mean: 1999.68 seconds (33.32 minutes)<br>
Standard deviation: 759.20 seconds<br>
Variance: 576382.01 seconds<br>

On Saturdays without rain, the mean duration of rides is approximately 1999.68 seconds, with a standard deviation of 759.20 seconds. This large variance of 576382.01 seconds² suggests considerable variability in ride durations around the mean.


**Rainy Saturdays:**
Mean: 2427.21 seconds (40.45 minutes) <br>
Standard deviation: 721.31 seconds<br>
Variance: 520294.09 seconds<br>

In contrast, rainy Saturdays show slightly longer mean ride durations, approximately 2427.21 seconds. Rainy Saturdays have a standard deviation of 721.31 seconds, slightly smaller than Saturdays without rain, suggesting more consistency. The variance for rainy Saturdays is also slightly lower at 520294.09 seconds² compared to Saturdays without rain.

When comparing the average durations in minutes, it's clear that Rainy Saturdays have longer rides, lasting approximately 40.45 minutes compared to 33.32 minutes on Non-Rainy Saturdays. This difference of approximately 13 extra minutes can feel significantly longer to passengers, especially considering that the average ride should take approximately 34.52 minutes.

Overall, there seems to be great variability between rides regardless of the weather on Saturdays, likely due to factors such as traffic conditions, construction, or peak hours for ride shares. Both non-rainy Saturdays and rainy Saturdays have variances much larger than the entire dataset mean of 2071.73 seconds, indicating a wide range of ride durations. Conducting a t-test will provide a better understanding of whether there is a significant difference between the two variables, helping determine if rain truly affects ride durations significantly.



### Conducting Hypothesis Testing

Since we're comparing the means of two independent samples (rides duration on rainy Saturdays vs. rides duration on non-rainy Saturdays), we can use the independent two-sample t-test and we will assign the significance level to 0.5, assuming equal variances from the tests done above.

Null Hypothesis (H0):
The average duration of rides from the Loop to O'Hare International Airport is the same on rainy Saturdays as on non-rainy Saturdays.

Alternative Hypothesis (H1):
The average duration of rides from the Loop to O'Hare International Airport differs on rainy Saturdays compared to non-rainy Saturdays.

In [ ]:
# Assigning alpha to 0.05
alpha = 0.05

# T-statictic and p-value
t_statistics, p_value = stats.ttest_ind(no_rain_saturdays, rain_saturdays, equal_var=True)

# Print t-statistic and p-value
print("T-statistic:", t_statistics)
print("P-value:", p_value)

# Perform hypothesis testing
if p_value < alpha:
    print("Reject the null hypothesis. The average duration of rides from the Loop to O'Hare International Airport differs on rainy Saturdays compared to non-rainy Saturdays.")
else:
    print("Fail to reject the null hypothesis. The average duration of rides from the Loop to O'Hare International Airport is the same on rainy Saturdays as on non-rainy Saturdays.")
    

: 

In [ ]:
#Being conservative and performing the test with equal_var=False as well since the Levene test was pretty close to 0.5

# Assigning alpha to 0.05
alpha = 0.05

# T-statictic and p-value
t_statistics, p_value = stats.ttest_ind(no_rain_saturdays, rain_saturdays, equal_var=False)

# Print t-statistic and p-value
print("T-statistic:", t_statistics)
print("P-value:", p_value)

# Perform hypothesis testing
if p_value < alpha:
    print("Reject the null hypothesis. The average duration of rides from the Loop to O'Hare International Airport differs on rainy Saturdays compared to non-rainy Saturdays.")
else:
    print("Fail to reject the null hypothesis. The average duration of rides from the Loop to O'Hare International Airport is the same on rainy Saturdays as on non-rainy Saturdays.")
    

: 

# Final Conclusions

As stated in the section above, **we reject the null hypothesis**. The average duration of rides from the Loop to O'Hare International Airport differs on rainy Saturdays compared to non-rainy Saturdays. When comparing the average durations in minutes, it's clear that Rainy Saturdays have longer rides, lasting approximately 40.45 minutes compared to 33.32 minutes on Non-Rainy Saturdays, while the average ride takes approximately 34.52 minutes.


**The Hypotheses Tested:** <br>
Null Hypothesis (H0): The average duration of rides from the Loop to O'Hare International Airport is the same on rainy Saturdays as on non-rainy Saturdays.

Alternative Hypothesis (HA): The average duration of rides from the Loop to O'Hare International Airport differs on rainy Saturdays compared to non-rainy Saturdays.



**Test Summary**<br>
By conducting Levene's test for equality of variances, I assessed whether the variances of the two groups (no_rain_saturdays and rain_saturdays) are significantly different. The p-value from Levene's test was greater than the significance level of 0.05 (p = 0.53), indicating no significant difference in variances between the groups. Therefore, I assumed equal variances and conducted a two-sample t-test with equal_var=True.

The t-test resulted in the following:
T-statistic: -6.946177714041499
P-value: 6.517970327099473e-12

The results led to the rejection of the null hypothesis, indicating that the average duration of rides from the Loop to O'Hare International Airport differs significantly between rainy Saturdays and non-rainy Saturdays.

Although the Levene test did not show a substantial difference in variances, I opted on the cautious side due to the small sample size (only Saturdays from November 2017). To ensure robustness, I also performed a Welch's t-test, assuming equal_var=False with the same significance level of 0.05.

The results from the Welch's test were as follows:
T-statistic: -7.186034288068629
P-value: 6.738994326108734e-12

Once again, the null hypothesis was rejected, confirming that the average duration of rides from the Loop to O'Hare International Airport differs significantly between rainy Saturdays and non-rainy Saturdays.



**Limitations**<br>
Although both tests supported the rejection of the null hypothesis, there are limitations to consider. One major limitation is the small sample size, limited to only Saturdays in November. This constraint may introduce biases or inaccuracies, as factors such as specific construction projects or Thanksgiving travel patterns could skew the data. To obtain a more accurate assessment of the significance of the observed differences, a larger sample size spanning various dates throughout the year, with consideration for different weather conditions, would be necessary.

Overall, while the results provide insights into the differences in ride durations based on weather conditions, caution is warranted in interpreting the findings due to the limitations of the dataset.